In [10]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("nominations.db")

schema = conn.execute("pragma table_info(nominations);")
first_ten = conn.execute("SELECT * FROM nominations LIMIT 10;")

for row in schema:
    print(row)

for row in first_ten:
    print(row)

(0, u'Year', u'INTEGER', 0, None, 0)
(1, u'Category', u'TEXT', 0, None, 0)
(2, u'Nominee', u'TEXT', 0, None, 0)
(3, u'Won?', u'INTEGER', 0, None, 0)
(4, u'Movie', u'TEXT', 0, None, 0)
(5, u'Character', u'TEXT', 0, None, 0)
(2010, u'Actor -- Leading Role', u'Javier Bardem', 0, u'Biutiful', u'Uxbal')
(2010, u'Actor -- Leading Role', u'Jeff Bridges', 0, u'True Grit', u'Rooster Cogburn')
(2010, u'Actor -- Leading Role', u'Jesse Eisenberg', 0, u'The Social Network', u'Mark Zuckerberg')
(2010, u'Actor -- Leading Role', u'Colin Firth', 1, u"The King's Speech", u'King George VI')
(2010, u'Actor -- Leading Role', u'James Franco', 0, u'127 Hours', u'Aron Ralston')
(2010, u'Actor -- Supporting Role', u'Christian Bale', 1, u'The Fighter', u'Dicky Eklund')
(2010, u'Actor -- Supporting Role', u'John Hawkes', 0, u"Winter's Bone", u'Teardrop')
(2010, u'Actor -- Supporting Role', u'Jeremy Renner', 0, u'The Town', u'James Coughlin')
(2010, u'Actor -- Supporting Role', u'Mark Ruffalo', 0, u'The Kids Are 

### Creating the ceremonies table

In [16]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

create_query = "CREATE TABLE ceremonies ( \
                id integer PRIMARY KEY, \
                Year integer, \
                Host text);"

conn.execute(create_query)

insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?, ?);"

conn.executemany(insert_query, years_hosts)

In [17]:
for row in conn.execute("SELECT * FROM ceremonies;"):
    print(row)

(1, 2010, u'Steve Martin')
(2, 2009, u'Hugh Jackman')
(3, 2008, u'Jon Stewart')
(4, 2007, u'Ellen DeGeneres')
(5, 2006, u'Jon Stewart')
(6, 2005, u'Chris Rock')
(7, 2004, u'Billy Crystal')
(8, 2003, u'Steve Martin')
(9, 2002, u'Whoopi Goldberg')
(10, 2001, u'Steve Martin')
(11, 2000, u'Billy Crystal')


### Foreign key constraints

In [19]:
conn.execute("PRAGMA foreign_keys = ON;")

### Setting up one-to-many

In [59]:
query = """SELECT ceremonies.id, nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations."won?"
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;"""

results = conn.execute(query).fetchall()

for row in results:
    print(row)

(1, u'Actor -- Leading Role', u'Javier Bardem', u'Biutiful', u'Uxbal', 0)
(1, u'Actor -- Leading Role', u'Jeff Bridges', u'True Grit', u'Rooster Cogburn', 0)
(1, u'Actor -- Leading Role', u'Jesse Eisenberg', u'The Social Network', u'Mark Zuckerberg', 0)
(1, u'Actor -- Leading Role', u'Colin Firth', u"The King's Speech", u'King George VI', 1)
(1, u'Actor -- Leading Role', u'James Franco', u'127 Hours', u'Aron Ralston', 0)
(1, u'Actor -- Supporting Role', u'Christian Bale', u'The Fighter', u'Dicky Eklund', 1)
(1, u'Actor -- Supporting Role', u'John Hawkes', u"Winter's Bone", u'Teardrop', 0)
(1, u'Actor -- Supporting Role', u'Jeremy Renner', u'The Town', u'James Coughlin', 0)
(1, u'Actor -- Supporting Role', u'Mark Ruffalo', u'The Kids Are All Right', u'Paul', 0)
(1, u'Actor -- Supporting Role', u'Geoffrey Rush', u"The King's Speech", u'Lionel Logue', 0)
(1, u'Actress -- Leading Role', u'Annette Bening', u'The Kids Are All Right', u'Nic', 0)
(1, u'Actress -- Leading Role', u'Nicole Kidman

In [42]:
create_query = """CREATE TABLE nominations_two (
                  id integer PRIMARY KEY, 
                  category text, 
                  nominee text, 
                  movie text,
                  character text,
                  won integer,
                  ceremony_id integer, 
                  FOREIGN KEY(ceremony_id) REFERENCES ceremony(id)
                  );
"""

conn.execute(create_query)

In [66]:
insert_query = "INSERT INTO nominations_two (ceremony_id, category, nominee, movie, character, won) VALUES (?, ?, ?, ?, ?, ?);"

conn.executemany(insert_query, results)

In [69]:
for row in conn.execute("SELECT * FROM nominations_two LIMIT 5;"):
    print(row)

(1, u'Actor -- Leading Role', u'Javier Bardem', u'Biutiful', u'Uxbal', 0, 1)
(2, u'Actor -- Leading Role', u'Jeff Bridges', u'True Grit', u'Rooster Cogburn', 0, 1)
(3, u'Actor -- Leading Role', u'Jesse Eisenberg', u'The Social Network', u'Mark Zuckerberg', 0, 1)
(4, u'Actor -- Leading Role', u'Colin Firth', u"The King's Speech", u'King George VI', 1, 1)
(5, u'Actor -- Leading Role', u'James Franco', u'127 Hours', u'Aron Ralston', 0, 1)


In [74]:
# Drop and rename table

conn.execute("DROP TABLE nominations;")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations;")

In [77]:
conn.execute("SELECT * FROM nominations LIMIT 5;").fetchall()

[(1, u'Actor -- Leading Role', u'Javier Bardem', u'Biutiful', u'Uxbal', 0, 1),
 (2,
  u'Actor -- Leading Role',
  u'Jeff Bridges',
  u'True Grit',
  u'Rooster Cogburn',
  0,
  1),
 (3,
  u'Actor -- Leading Role',
  u'Jesse Eisenberg',
  u'The Social Network',
  u'Mark Zuckerberg',
  0,
  1),
 (4,
  u'Actor -- Leading Role',
  u'Colin Firth',
  u"The King's Speech",
  u'King George VI',
  1,
  1),
 (5,
  u'Actor -- Leading Role',
  u'James Franco',
  u'127 Hours',
  u'Aron Ralston',
  0,
  1)]

### Creating a join table

In [78]:
conn.execute("""CREATE TABLE movies (
id integer PRIMARY KEY, 
movie text);"""
            )

conn.execute("""CREATE TABLE actors (
id integer PRIMARY KEY, 
actor text);""")

conn.execute("""CREATE TABLE movies_actors (
id integer PRIMARY KEY, 
movie_id integer, 
actor_id integer, 
FOREIGN KEY(movie_id) REFERENCES movie(id), 
FOREIGN KEY(actor_id) REFERENCES actor(id));""")

### Insert records into join tables

In [80]:
movies = conn.execute("SELECT DISTINCT movie FROM nominations").fetchall()

print(movies)

insert_query = "INSERT INTO movies (movie) VALUES (?);"

conn.executemany(insert_query, movies)

conn.execute("SELECT * FROM movies LIMIT 10;").fetchall()

[(u'127 Hours',), (u'21 Grams',), (u'A Beautiful Mind',), (u'A History of Violence',), (u'A Single Man',), (u'About Schmidt',), (u'Adaptation',), (u'Ali',), (u'Almost Famous',), (u'American Gangster',), (u'An Education',), (u'Animal Kingdom',), (u'Atonement',), (u'Away from Her',), (u'Babel',), (u'Before Night Falls',), (u'Being Julia',), (u'Billy Elliot',), (u'Biutiful',), (u'Black Swan',), (u'Blood Diamond',), (u'Blue Valentine',), (u"Bridget Jones's Diary",), (u'Brokeback Mountain',), (u'Capote',), (u'Cast Away',), (u'Catch Me If You Can',), (u'Changeling',), (u"Charlie Wilson's War",), (u'Chicago',), (u'Chocolat',), (u'Cinderella Man',), (u'Closer',), (u'Cold Mountain',), (u'Collateral',), (u'Crash',), (u'Crazy Heart',), (u'Doubt',), (u'Dreamgirls',), (u'Eastern Promises',), (u'Elizabeth: The Golden Age',), (u'Erin Brockovich',), (u'Eternal Sunshine of the Spotless Mind',), (u'Far from Heaven',), (u'Finding Neverland',), (u'Frida',), (u'Frost/Nixon',), (u'Frozen River',), (u'Gangs 

[(1, u'127 Hours'),
 (2, u'21 Grams'),
 (3, u'A Beautiful Mind'),
 (4, u'A History of Violence'),
 (5, u'A Single Man'),
 (6, u'About Schmidt'),
 (7, u'Adaptation'),
 (8, u'Ali'),
 (9, u'Almost Famous'),
 (10, u'American Gangster')]

In [81]:
actors = conn.execute("SELECT DISTINCT nominee FROM nominations").fetchall()

print(actors)

insert_query = "INSERT INTO actors (actor) VALUES (?);"

conn.executemany(insert_query, actors)

conn.execute("SELECT * FROM actors LIMIT 10;").fetchall()

[(u'Abigail Breslin',), (u'Adriana Barraza',), (u'Adrien Brody',), (u'Alan Alda',), (u'Alan Arkin',), (u'Albert Finney',), (u'Alec Baldwin',), (u'Amy Adams',), (u'Amy Ryan',), (u'Angelina Jolie',), (u'Anna Kendrick',), (u'Anne Hathaway',), (u'Annette Bening',), (u'Ben Kingsley',), (u'Benicio Del Toro',), (u'Bill Murray',), (u'Brad Pitt',), (u'Carey Mulligan',), (u'Casey Affleck',), (u'Catalina Sandino Moreno',), (u'Cate Blanchett',), (u'Catherine Keener',), (u'Catherine Zeta-Jones',), (u'Charlize Theron',), (u'Chris Cooper',), (u'Christian Bale',), (u'Christoph Waltz',), (u'Christopher Plummer',), (u'Christopher Walken',), (u'Clint Eastwood',), (u'Clive Owen',), (u'Colin Firth',), (u'Daniel Day-Lewis',), (u'David Strathairn',), (u'Denzel Washington',), (u'Diane Keaton',), (u'Diane Lane',), (u'Djimon Hounsou',), (u'Don Cheadle',), (u'Ed Harris',), (u'Eddie Murphy',), (u'Ellen Burstyn',), (u'Ellen Page',), (u'Ethan Hawke',), (u'Felicity Huffman',), (u'Forest Whitaker',), (u'Frances McDor

[(1, u'Abigail Breslin'),
 (2, u'Adriana Barraza'),
 (3, u'Adrien Brody'),
 (4, u'Alan Alda'),
 (5, u'Alan Arkin'),
 (6, u'Albert Finney'),
 (7, u'Alec Baldwin'),
 (8, u'Amy Adams'),
 (9, u'Amy Ryan'),
 (10, u'Angelina Jolie')]

In [89]:
movie_actor = conn.execute("""
SELECT movies.id, actors.id 
FROM movies, actors, nominations
WHERE movies.movie = nominations.movie
AND actors.actor = nominations.nominee""").fetchall()

# 220 rows
print(movie_actor)

insert_query = "INSERT INTO movies_actors (movie_id, actor_id) VALUES (?, ?);"

conn.executemany(insert_query, movie_actor)

conn.execute("SELECT * FROM movies_actors LIMIT 10;").fetchall()

[(1, 66), (2, 15), (2, 115), (3, 70), (3, 133), (4, 158), (5, 32), (6, 62), (6, 92), (7, 25), (7, 108), (7, 117), (8, 81), (8, 156), (9, 47), (9, 90), (10, 132), (11, 18), (12, 63), (13, 138), (14, 87), (15, 2), (15, 130), (16, 68), (17, 13), (18, 88), (19, 68), (20, 116), (21, 38), (21, 97), (22, 111), (23, 128), (24, 55), (24, 65), (24, 111), (25, 22), (25, 124), (26, 149), (27, 29), (28, 10), (29, 124), (30, 23), (30, 78), (30, 125), (30, 128), (31, 84), (31, 89), (32, 120), (33, 31), (33, 116), (34, 83), (34, 128), (35, 67), (36, 106), (37, 69), (37, 98), (38, 8), (38, 108), (38, 124), (38, 154), (39, 41), (39, 71), (40, 153), (41, 21), (42, 6), (42, 85), (43, 91), (44, 86), (45, 80), (46, 135), (47, 48), (48, 107), (49, 33), (50, 77), (50, 133), (51, 9), (52, 34), (53, 56), (53, 99), (54, 134), (55, 39), (55, 142), (56, 14), (56, 140), (57, 145), (58, 139), (59, 21), (60, 38), (60, 136), (61, 102), (61, 141), (61, 150), (62, 151), (63, 27), (64, 53), (65, 105), (65, 114), (66, 75)

[(1, 1, 66),
 (2, 2, 15),
 (3, 2, 115),
 (4, 3, 70),
 (5, 3, 133),
 (6, 4, 158),
 (7, 5, 32),
 (8, 6, 62),
 (9, 6, 92),
 (10, 7, 25)]

In [90]:
conn.close()